In [42]:
#google_document_xlsx = "https://docs.google.com/spreadsheets/d/1riSLCvdlQ8kC5DshtqlRAsVEa7R7N2kD/export?format=xlsx"
google_document_xlsx = "https://docs.google.com/spreadsheets/d/1VaDe0D14U3fZAWtx0wMXtSUT1_EDL5mUaNuGjxJ6y04/export?format=xlsx"

In [43]:
!wget {google_document_xlsx} -O /tmp/prices_scans.xlsx

--2025-04-24 10:51:13--  https://docs.google.com/spreadsheets/d/1VaDe0D14U3fZAWtx0wMXtSUT1_EDL5mUaNuGjxJ6y04/export?format=xlsx
Resolving docs.google.com (docs.google.com)... 216.58.211.238, 2a00:1450:4026:808::200e
Connecting to docs.google.com (docs.google.com)|216.58.211.238|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: https://doc-10-24-sheets.googleusercontent.com/export/54bogvaave6cua4cdnls17ksc4/i2i90ncrn248708asauan0pcso/1745491870000/116312231347809998123/*/1VaDe0D14U3fZAWtx0wMXtSUT1_EDL5mUaNuGjxJ6y04?format=xlsx [following]
--2025-04-24 10:51:13--  https://doc-10-24-sheets.googleusercontent.com/export/54bogvaave6cua4cdnls17ksc4/i2i90ncrn248708asauan0pcso/1745491870000/116312231347809998123/*/1VaDe0D14U3fZAWtx0wMXtSUT1_EDL5mUaNuGjxJ6y04?format=xlsx
Resolving doc-10-24-sheets.googleusercontent.com (doc-10-24-sheets.googleusercontent.com)... 216.58.211.225, 2a00:1450:4026:808::2001
Connecting to doc-10-24-sheets.googleusercontent.com

In [49]:
import pandas as pd

In [51]:
df = pd.read_excel('/tmp/prices_scans.xlsx', header=1)

In [52]:
df.set_index("Номер картинки",  inplace=True)

In [6]:
df.head(4)

,Цена правильная,Цена нейросеть,Ошибка,Строка 1,Строка 2,Строка 3,Строка 4,Строка 5
Номер картинки,,,,,,,,
1,54.99,54.99,0.0,Вода СВЯТОЙ ИСТОЧНИК,1.0л,природная питьевая негаз.ПЭТ,NaN,NaN
2,59.99,59.99,0.0,Вода Святой источник,"0,75л",негазированная спорт,NaN,NaN
3,51.99,51.99,0.0,Вода ШИШКИН ЛЕС,питьевая 1л,негазированная,NaN,NaN
4,24.99,24.99,0.0,Вода КРАСНАЯ ЦЕНА,"1,5л",негазированная,NaN,NaN


## На основе записей в чеке

In [8]:
import re
from collections import Counter

def generate_words_from_phrases(phrases, len_min=1, do_sort=True):
    word_counter = Counter()

    for phrase in phrases:
        if not isinstance(phrase, str):
            continue  # пропускаем нестроковые значения

        
        words = re.split(r"[ \t\n\r\f\v!?;:()\"«»—–\/]+", phrase.upper().replace("Ё", "Е"))
        words = list(filter(None, words))  # убираем пустые строки, если они есть
        
        # Оставляем только слова без цифр
        filtered_words = [
            word for word in words
            if len(word) > len_min and not re.search(r"\d", word)
        ]
        word_counter.update(filtered_words)

    if do_sort:
        return sorted(word_counter.items(), key=lambda x: x[1], reverse=True)
    else:
        return word_counter.items()


In [10]:
# Здесь у нас все фразы из таблицы
phrases = pd.concat([df["Строка 1"].dropna(), df["Строка 2"].dropna()]).astype(str).unique()


In [11]:
sorted_word_counters = generate_words_from_phrases(phrases)


In [12]:
new_df = pd.DataFrame(sorted_word_counters, columns=["word", "count"])
new_df

,word,count
0,ВИНО,23
1,ПИВО,21
2,НАПИТОК,20
3,СМАК,18
4,СЫР,16
...,...,...
987,ЛУКОМ,1
988,СПЕЛЫЙ,1
989,ОБЖАРЕННЫЕ,1
990,КИТ,1


In [13]:
new_df.drop_duplicates(subset=['word'], inplace=True)


In [14]:
new_df[new_df['word'].str.contains("фруто", case=False, na=False)]

,word,count
17,ФРУТОНЯНЯ,11
117,ФРУТО,3
199,ФРУТО-НЯНЯ,2
701,ФРУТО-,1


In [13]:
del(new_df['count'])
new_df.reset_index(drop=True, inplace=True)
new_df.to_csv('/home/jovyan/work/words.txt', index=False, encoding='utf-8', header=False)

##  Предложенное Владимиром

In [23]:
xls = pd.ExcelFile('/tmp/prices_scans.xlsx')
print(xls.sheet_names)

['Лист1', 'База слов 1 с ценников 2 строки', 'База слов 2 с ценников 3 строки', 'База слов 3 Купер', 'Лист3', 'Лист4', 'Лист4 (копия)', 'Проверка распознавания']


In [30]:
df = pd.read_excel('/tmp/prices_scans.xlsx', sheet_name="База слов 1 с ценников 2 строки")
df.head()

,Слово,Частота,Длина,Unnamed: 3,Слова собраны с первых двух строк ценников (крупный текст)
0,ТРУСИКИ-ПОДГУЗНИКИ,2,18,NaN,NaN
1,КРАСНОС.-АНЧЕЛОТТА,1,18,NaN,NaN
2,БАЛЬЗАМ-ОПОЛАСКИВ.,1,18,NaN,NaN
3,ОРЕХОВО-ФРУКТОВОЕ,2,17,NaN,NaN
4,ШАМПУНЬ/БАЛ-ОПОЛ.,1,17,NaN,NaN


In [31]:
df_kupper = pd.read_excel('/tmp/prices_scans.xlsx', sheet_name="База слов 3 Купер", header=None)
df.head()

,Слово,Частота,Длина,Unnamed: 3,Слова собраны с первых двух строк ценников (крупный текст)
0,ТРУСИКИ-ПОДГУЗНИКИ,2,18,NaN,NaN
1,КРАСНОС.-АНЧЕЛОТТА,1,18,NaN,NaN
2,БАЛЬЗАМ-ОПОЛАСКИВ.,1,18,NaN,NaN
3,ОРЕХОВО-ФРУКТОВОЕ,2,17,NaN,NaN
4,ШАМПУНЬ/БАЛ-ОПОЛ.,1,17,NaN,NaN


In [36]:
df_combined = pd.concat([df.iloc[:,0], df_kupper.iloc[:,0]], ignore_index=True).to_frame(name="Combined_Column")
df_combined

,Combined_Column
0,ТРУСИКИ-ПОДГУЗНИКИ
1,КРАСНОС.-АНЧЕЛОТТА
2,БАЛЬЗАМ-ОПОЛАСКИВ.
3,ОРЕХОВО-ФРУКТОВОЕ
4,ШАМПУНЬ/БАЛ-ОПОЛ.
...,...
12373,LORI
12374,НЯМА
12375,NUTZ
12376,YES!


In [ ]:
# Удаляем пустые строки
# df_combined.dropna(inplace=True)

In [39]:
df_combined.reset_index(drop=True, inplace=True)
df_combined.to_csv('/home/jovyan/work/words.txt', index=False, encoding='utf-8', header=False)

# Загружаем результаты распознанвания для анализа

In [53]:
saved_xlsx = "https://docs.google.com/spreadsheets/d/1E92a3fsSQM_Lp13hjONme5aGaWTxRYcWBQussVfE8yE/export?format=xlsx"


In [54]:
!wget -O results.xlsx $saved_xlsx

--2025-04-24 10:53:16--  https://docs.google.com/spreadsheets/d/1E92a3fsSQM_Lp13hjONme5aGaWTxRYcWBQussVfE8yE/export?format=xlsx
Resolving docs.google.com (docs.google.com)... 216.58.211.238, 2a00:1450:4026:808::200e
Connecting to docs.google.com (docs.google.com)|216.58.211.238|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: https://doc-08-1s-sheets.googleusercontent.com/export/54bogvaave6cua4cdnls17ksc4/cfjb0moh3j69dh4v9sio4thpdg/1745491995000/117963286932567830463/*/1E92a3fsSQM_Lp13hjONme5aGaWTxRYcWBQussVfE8yE?format=xlsx [following]
--2025-04-24 10:53:17--  https://doc-08-1s-sheets.googleusercontent.com/export/54bogvaave6cua4cdnls17ksc4/cfjb0moh3j69dh4v9sio4thpdg/1745491995000/117963286932567830463/*/1E92a3fsSQM_Lp13hjONme5aGaWTxRYcWBQussVfE8yE?format=xlsx
Resolving doc-08-1s-sheets.googleusercontent.com (doc-08-1s-sheets.googleusercontent.com)... 216.58.211.225, 2a00:1450:4026:808::2001
Connecting to doc-08-1s-sheets.googleusercontent.com

In [55]:
results_df = pd.read_excel('results.xlsx') 

In [56]:
results_df

,file_name,text,elapsed_time,corrected_text,elapsed_time_corrected,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,1,Вода СВЯТОЙ ИСТОЧНИК\n1.0л\nПригодна пить всем,5.1053,ВОДА СВЯТОЙ ИСТОЧНИК\n1 0Л\nПРИРОДНАЯ ПИТЬ ВСЕМ,0.0004,NaN,NaN,NaN
1,2,"Вода Святой источник\n0,75л\nнегазированная вода",4.2579,ВОДА СВЯТОЙ ИСТОЧНИК\n0 75Л\nНЕГАЗИРОВАННАЯ ВОДА,0.0004,NaN,NaN,NaN
2,3,Вода ШИШКИН ЛЕС\nпитьевая 1л\nАО «Заирoколина»,5.2103,ВОДА ШИШКИН ЛЕС\nПИТЬЕВАЯ 1Л\nАО ЗАИРOКОЛИНА,0.0006,NaN,NaN,NaN
3,4,Цена красная,3.4990,ЦЕНА КРАСНАЯ,0.0001,NaN,NaN,NaN
4,5,Вода Святой Источник\n1.5л\nх.почечен. без сах...,5.0172,ВОДА СВЯТОЙ ИСТОЧНИК\n1 5Л\nХ ПЕЧЕН БЕЗ САХ ПИЛ Б,0.0005,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
993,1004,Томаты,3.3057,ТОМАТЫ,0.0001,NaN,NaN,NaN
994,1005,Ароматизированный,3.4513,ГЛАЗИРОВАННЫЙ,0.0003,NaN,NaN,NaN
995,1006,Нектарины\n1кг\nузбекские,4.2026,НЕКТАРИНЫ\n1КГ\nУЗБЕКСКИЕ,0.0005,NaN,NaN,NaN
996,1007,Имбирь\n100г,3.9998,ИМБИРЬ\n100Г,0.0001,NaN,NaN,NaN


In [57]:
results_df.set_index("file_name", inplace=True)

In [62]:
df_strings3 = df.copy()

In [63]:
df_strings3 = df_strings3[["Строка 1", "Строка 2", "Строка 3"]]

In [64]:
df_strings3

,Строка 1,Строка 2,Строка 3
Номер картинки,,,
1,Вода СВЯТОЙ ИСТОЧНИК,1.0л,природная питьевая негаз.ПЭТ
2,Вода Святой источник,"0,75л",негазированная спорт
3,Вода ШИШКИН ЛЕС,питьевая 1л,негазированная
4,Вода КРАСНАЯ ЦЕНА,"1,5л",негазированная
5,Вода СВЯТОЙ ИСТОЧНИК,"1,5л","ключевая, без газа пл/б"
...,...,...,...
1004,Томаты 1кг,NaN,NaN
1005,Апельсины 1кг,NaN,NaN
1006,Нектарины 1кг,узбекские,NaN


In [65]:
df_strings3["merged"] = df_strings3.apply(
    lambda row: "\n".join([str(row["Строка 1"]), str(row["Строка 2"]), str(row["Строка 3"])]),
    axis=1
)

In [68]:
results_df['original_text'] = results_df.index.map(df_strings3['merged'])
results_df.head()

,text,elapsed_time,corrected_text,elapsed_time_corrected,Unnamed: 5,Unnamed: 6,Unnamed: 7,original_text
file_name,,,,,,,,
1,Вода СВЯТОЙ ИСТОЧНИК\n1.0л\nПригодна пить всем,5.1053,ВОДА СВЯТОЙ ИСТОЧНИК\n1 0Л\nПРИРОДНАЯ ПИТЬ ВСЕМ,0.0004,NaN,NaN,NaN,Вода СВЯТОЙ ИСТОЧНИК\n1.0л\nприродная питьевая...
2,"Вода Святой источник\n0,75л\nнегазированная вода",4.2579,ВОДА СВЯТОЙ ИСТОЧНИК\n0 75Л\nНЕГАЗИРОВАННАЯ ВОДА,0.0004,NaN,NaN,NaN,"Вода Святой источник\n0,75л\nнегазированная спорт"
3,Вода ШИШКИН ЛЕС\nпитьевая 1л\nАО «Заирoколина»,5.2103,ВОДА ШИШКИН ЛЕС\nПИТЬЕВАЯ 1Л\nАО ЗАИРOКОЛИНА,0.0006,NaN,NaN,NaN,Вода ШИШКИН ЛЕС\nпитьевая 1л\nнегазированная
4,Цена красная,3.4990,ЦЕНА КРАСНАЯ,0.0001,NaN,NaN,NaN,"Вода КРАСНАЯ ЦЕНА\n1,5л\nнегазированная"
5,Вода Святой Источник\n1.5л\nх.почечен. без сах...,5.0172,ВОДА СВЯТОЙ ИСТОЧНИК\n1 5Л\nХ ПЕЧЕН БЕЗ САХ ПИЛ Б,0.0005,NaN,NaN,NaN,"Вода СВЯТОЙ ИСТОЧНИК\n1,5л\nключевая, без газа..."


In [70]:
results_df.to_excel("results_with_original_text.xlsx", index=True)